# Импорты

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json

# Чтение исходных данных

In [27]:
DATAPATH = './data/'

In [28]:
train_file = DATAPATH + 'train.json'
test_file = DATAPATH + 'test.json'

In [29]:
sample = pd.read_csv(DATAPATH + 'sample.csv')
sample.head()

,id,sentiment
0,0,neutral
1,1,positive
2,2,negative
3,3,positive
4,4,neutral


In [30]:
def get_df(json_filename):
    with open(json_filename) as json_file:
        temp = json.load(json_file)
        return pd.DataFrame.from_records(temp)

In [31]:
train = get_df(train_file)
train.head()

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative


In [32]:
test = get_df(test_file)
test.head()

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4
